## Chapter 9
# Virtual Musical Instruments (cont.)

In [11]:
import numpy as np

import sys
sys.path.append('../')

from InterpolatingDelayLine import InterpolatingDelayLine
from Filters import TwoZeroFilter, PoleZeroFilter, IirFilter

sample_rate = 44100
fs = sample_rate

class PianoString:
    def __init__(self, lowest_frequency):
        # optional gain knob - should be stable at 1.0 but I found feedback for some notes (e.g. G3)
        self.loop_gain = 1.0
        self.loop_gain_target = 1.0
        self.filter_delay = 1 # delay of the two zero loop filters (samples)
        self.detuning = 0.998 # Note: Has a massive impact on the sustain sound as well
        self.coupling_amount = 0.01
        
        # Instantiate delay line by the longest length delay
        # Compensated by the delay from filter(s)
        self.length = int(sample_rate / lowest_frequency + self.filter_delay)
        
        self.last_frequency = lowest_frequency
        self.last_length = float(self.length)
        
        # Set up the string (optional correction in length to compensate for filtering)
        self.delay_line_v = InterpolatingDelayLine(self.length + 1) # init with max length
        self.delay_line_v.set_delay_samples(self.last_length)
        self.delay_line_h = InterpolatingDelayLine(self.length + 1)
        self.delay_line_h.set_delay_samples(self.last_length)
        
        self.pluck_position = 1/8
        self.pluck_comb_delay = InterpolatingDelayLine(self.length)
        self.set_pluck_position(self.pluck_position)
        
        self.t60_initial = 3.0
        self.t60_sustain = 9.0
        self.string_brightness = 0.5
        self.loop_filter_v = TwoZeroFilter()
        self.loop_filter_h = TwoZeroFilter()
        self.set_loop_filters()
        
        self.dc_block_filter = PoleZeroFilter()
        self.dc_block_filter.set_block_zero()
        
        self.clear()
    
    def clear(self):
        self.out_v = 0.0
        self.out_h = 0.0
        self.out_c1 = 0.0
        self.out_c2 = 0.0
        
        self.delay_line_v.clear()
        self.loop_filter_v.clear()
        self.delay_line_h.clear()
        self.loop_filter_h.clear()
        self.pluck_comb_delay.clear()
    
    def set_frequency(self, frequency):
        self.last_frequency = frequency
        self.last_length = sample_rate / self.last_frequency
        # delay lines compensated by filter delays
        self.delay_line_v.set_delay_samples(self.last_length / self.detuning - self.filter_delay)
        self.delay_line_h.set_delay_samples(self.last_length * self.detuning - self.filter_delay)
        self.set_loop_filters()
        
    def set_loop_gain(self, gain):
        # I added some linear smoothing since rapidly changing loop gain was causing comb filter buzzing
        self.loop_gain_target = gain
    
    def set_detuning(self, detuning):
        self.detuning = detuning
        
    def set_brightness(self, brightness):
        self.string_brightness = brightness
        self.set_loop_filters()
    
    def set_pluck_position(self, pluck_position):
        self.pluck_position = pluck_position
        self.pluck_comb_delay.set_delay_samples(pluck_position * self.last_length)
    
    def set_t60_initial(self, t60):
        self.t60_initial = t60
        self.set_loop_filters()
    
    def set_t60_sustain(self, t60):
        self.t60_sustain = t60
        self.set_loop_filters()
    
    def set_loop_filters(self):
        B = self.string_brightness

        S = self.t60_initial
        g0 = np.exp(-6.91 / ((self.last_frequency / self.detuning) * S))
        b0 = g0 * (1 + B) / 2.0
        b1 = g0 * (1 - B) / 4.0
        self.loop_filter_v.set_coefficients(b1, b0, b1)

        S = self.t60_sustain
        g0 = np.exp(-6.91 / ((self.last_frequency * self.detuning) * S))
        b0 = g0 * (1 + B) / 2.0
        b1 = g0 * (1 - B) / 4.0
        self.loop_filter_h.set_coefficients(b1, b0, b1)
    
    def tick(self, in_sample):
        if self.loop_gain < self.loop_gain_target:
            self.loop_gain += 0.001
        elif self.loop_gain > self.loop_gain_target:
            self.loop_gain -= 0.001

        gen_input = in_sample - self.pluck_comb_delay.tick(in_sample)
        
        # Coupling amount and coupled string system modeled after:
        # M. Aramaki, J. Bensa, L. Daudet, Ph. Guillemain, and R. Kronland-Martinet
        # "Resynthesis of Coupled Piano String Vibrations Based on Physical Modeling"
        # Journal of New Music Research 2001, Vol. 30, No. 3, pp. 213-226
        # Going through strings and loop filters, with some coupling (and loopGain)
        self.out_v = self.loop_filter_v.tick(self.loop_gain * self.delay_line_v.tick(gen_input + self.out_c2 + self.out_v))
        self.out_c1 = self.out_v * self.coupling_amount
        self.out_h = self.loop_filter_h.tick(self.loop_gain * self.delay_line_h.tick(self.out_c1 + self.out_h))
        self.out_c2 = self.dc_block_filter.tick(self.out_h * self.coupling_amount)
        
        return self.out_v # return signal from vertical string

In [8]:
from WavPlayer import WavPlayer

class CommutedPiano:
    def __init__(self):
        self.wave_done = True # whether excitation input is finished
        self.lowest_frequency = 27.5 # A0
        self.highest_frequency = 4186.0 # C8

        self.hammer_pulse = IirFilter()
        self.body_response = WavPlayer('soundboard.wav')
        self.piano_string = PianoString(self.lowest_frequency)
        self.current_frequency = self.lowest_frequency
        self.current_amplitude = 0.0
        self.string_input = 0

    def tick(self):
        # Tick out excitation filtered noise
        if not self.wave_done:
            self.string_input = self.hammer_pulse.tick(self.body_response.tick())
        
        out = self.piano_string.tick(self.string_input)
        out *= 1.0 # optional gain hacking
        
        if self.body_response.is_finished():
            self.wave_done = True
            self.string_input = 0.0
        
        return out

    def set_frequency(self, frequency):
        self.current_frequency = frequency
        self.piano_string.set_frequency(frequency)
    
    def note_on(self, frequency, amplitude):
        self.piano_string.set_loop_gain(1.0)
        self.set_frequency(frequency)
        self.current_amplitude = amplitude
        self.set_excitation_pulse()
        self.string_input = 0
        self.body_response.reset()
        self.wave_done = False
    
    def note_off(self, amplitude=1.0): # off-amplitude is not currently used
        # Damping coefficients range from 0.75 (A0) to 0.9 (A5 and above)
        # Julien Bensa, "Analyse et Synthese de sons de piano par modeles
        # physiques et de signaux", These de doctorat de l'universite de la
        # Mediterranee Aix-Marseille II, 2003, p. 140
        
        a5_frequency = 880.0
        if self.current_frequency <= a5_frequency:
            loop_scalar = (self.current_frequency - self.lowest_frequency) / (a5_frequency - self.lowest_frequency)
            loop_gain = 0.75 + loop_scalar * (0.9 - 0.75)
        else:
            loop_gain = 0.9
        
        self.piano_string.set_loop_gain(loop_gain)

    def set_excitation_pulse(self):
        # Anders Askenfelt and Erik Janson "From touch to string vibrations"
        # Five Lectures on the Acoustics of the Piano
        # http://www.speech.kth.se/music/5_lectures/askenflt/askenflt.html
        # All in ms units:
        min_duration_hf = 0.5 # ff; min duration at high register
        min_duration_lf = 2.0 # ff; min duration at low register
        max_duration_hf = 1.2 # pp; max duration at high register
        max_duration_lf = 4.0 # pp; max duration at low register
        velocity_scalar = self.current_amplitude

        frequency_scalar = 0.0
        if self.current_frequency < self.lowest_frequency:
            frequency_scalar = 0.0
        elif self.current_frequency > self.highest_frequency:
            frequency_scalar = 1.0
        else:
            frequency_scalar = (np.log10(self.current_frequency) - np.log10(self.lowest_frequency)) / (np.log10(self.highest_frequency) - np.log10(self.lowest_frequency))

        min_duration = min_duration_lf - frequency_scalar * (min_duration_lf - min_duration_hf)
        max_duration = max_duration_lf - frequency_scalar * (max_duration_lf - max_duration_hf)
        pulse_duration = max_duration - velocity_scalar * (max_duration - min_duration)
        pulse_length = int(pulse_duration * fs / 1000)
        pulse = 0.5 + 0.5 * np.cos(2 * np.pi * np.linspace(-0.5, 0.5, pulse_length))

        self.hammer_pulse.set_b_coefficients(pulse)
        self.hammer_pulse.set_gain(self.current_amplitude / pulse.sum()) # normalize so gain is 0 dB at DC
    
    def set_strike_position(self):
        # Striking positions range from 0.122 (A0) to 0.115 (A4) to 0.08 (C8)
        # Harold A. Conklin, Jr.
        # "Design and tone in the mechanoacoustic piano. Part I. Piano hammers
        # and tonal effects" Journal of the Acoustical Society of America
        # Vol. 99, No. 6, June 1996, p. 3293
        a4_frequency = 440.0
        if currentFreq <= a4_frequency:
            strike_scalar = (np.log10(self.current_frequency) - np.log10(self.lowest_frequency)) / (np.log10(a4_frequency) - np.log10(self.lowest_frequency))
            strike_position = 0.122 - strike_scalar * (0.122 - 0.115)
        else:
            strike_scalar = (self.highest_frequency - self.current_frequency) / (self.highest_frequency - a4_frequency)
            strike_position = 0.08 + strike_scalar * (0.115 - 0.08)

        self.piano_string.set_pluck_position(strike_position)

    def set_t60(self):
        # Initial t60s range from 15 seconds (A0) to 0.3 seconds (C8)
        # Sustained t60s range from 50 seconds (A0) to 0.3 seconds (C8)
        # Fletcher and Rossing "The Physics of Musical Instruments", 2nd edition
        # Springer-Verlag, New York, 1998, p. 384
        t60_scalar = (self.current_frequency - self.lowest_frequency) / (self.highest_frequency - self.lowest_frequency)
        # FIXME extract into function
        initial_t60 = 10.0 ** (np.log10(15.0) - t60_scalar * (np.log10(15.0) - np.log10(0.3)))
        sustain_t60 = 10.0 ** (np.log10(50.0) - t60_scalar * (np.log10(50.0) - np.log10(0.3)))

        self.piano_string.set_t60_initial(initial_t60)
        self.piano_string.set_t60_sustain(sustain_t60)

In [9]:
class PolyphonicCommutedPiano():
    def __init__(self, num_voices=10):
        self.voices = [CommutedPiano() for _ in range(num_voices)]
    
    # Convenience method to arpeggiate across notes,
    # with gaps between `note_on` events set by `note_delay_samples`
    def arpeggiate(self, frequencies, num_samples=fs*8, note_delay_samples=fs//12):
        out = np.zeros(num_samples)
        for i in range(num_samples):
            frequency_index = i//note_delay_samples
            if i % note_delay_samples == 0 and frequency_index < len(frequencies):
                frequency = frequencies[frequency_index]
                # TODO voice management - "lift" least-recent "finger" with `note_off` if we run out
                self.voices[frequency_index % len(self.voices)].note_on(frequency, 1.0)
            out[i] = self.tick()
        return out

    def tick(self):
        out = 0.0
        for voice in self.voices: # TODO string coupling
            out += voice.tick()

        self.previous_out = out
        return out

In [4]:
from IPython.display import Audio
sys.path.append('../musimathics')
from pitches import frequencies_for_note_labels

frequencies = frequencies_for_note_labels(['A2', 'C#3', 'E3',
                                           'A4', 'C#5', 'E5',
                                           'A5', 'C#6', 'E6']) # three major A triads
piano = PolyphonicCommutedPiano(num_voices=len(frequencies))

piano_arp_no_loop_filter = piano.arpeggiate(frequencies, note_delay_samples=fs//10)

Audio(piano_arp_no_loop_filter, rate=fs)

In [14]:
from pitches import frequency_for_note_label

a1_frequency = frequency_for_note_label('A#1')

single_voice = CommutedPiano()
num_out_samples = fs * 10
piano_striking_single_key = np.zeros(num_out_samples)
# Hit key every quarter second and release after a random short duration
note_on_samples = np.arange(0, fs * 6, fs // 4)
note_off_samples = note_on_samples + np.random.randint(low=fs//40, high=fs//6, size=note_on_samples.size)
# "hold down" last note until a second before the end
note_off_samples[-1] = num_out_samples - fs

for i in range(num_out_samples):
    if i in note_on_samples:
        amp = np.random.uniform(low=0.7, high=1.0) if i != note_on_samples[-1] else 1.0 # end with a loud note
        single_voice.note_on(a1_frequency, amp)
    elif i in note_off_samples:
        single_voice.note_off()

    piano_striking_single_key[i] = single_voice.tick()

Audio(piano_striking_single_key, rate=fs)

In [15]:
from scipy.io.wavfile import write as write_wav
write_wav('piano_arp_no_loop_filter.wav', rate=fs, data=piano_arp_no_loop_filter)
write_wav('piano_striking_single_key.wav', rate=fs, data=piano_striking_single_key)